# Evalute OVASO

In [1]:
import torch
import pandas as pd
import numpy as np
import time
import os
import random
import matplotlib.pyplot as plt

from datasets.utils import get_dataloaders
from models.model import get_model, eval_model
from models.utils import set_random_seed

## Set parameters

In [2]:
val_dir = './data/CXR/test/'
model_dir = './trained_models/'
out_dir = './results'

In [3]:
batch_size = 16
random_seed = 0

In [4]:
set_random_seed(random_seed)

## Get data loaders

In [5]:
dataloaders = get_dataloaders(val_dir, batch_size)

Validation dataset size: 300
['covid-19', 'normal', 'pneumonia']
######### Validation Dataset #########
covid-19 size: 100
normal size: 100
pneumonia size: 100


## Evaluate the model

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# mtype = 'MOVR'
mtype = 'OVR'

best_preds_dict, best_confidence_weights = None, None
best_acc, best_covid_threshold = 0.0, 0.0
if mtype == 'OVR':
    covid_ft, normal_ft, pneumonia_ft, \
    cov_nor_ft, cov_pneu_ft, nor_pneu_ft, criterion = \
        get_model(model_dir, device, mtype)
    preds_dict, acc = eval_model(dataloaders=dataloaders,
                                 covid_model=covid_ft,
                                 normal_model=normal_ft,
                                 pneumonia_model=pneumonia_ft,
                                 cov_nor_model=cov_nor_ft,
                                 cov_pneu_model=cov_pneu_ft,
                                 nor_pneu_model=nor_pneu_ft,
                                 criterion=criterion,
                                 batch_size=batch_size,
                                 device=device,
                                 out_dir=out_dir,
                                 mtype=mtype)
else: # Previous OVR
    covid_ft, nor_pneu_ft, criterion = get_model(model_dir, device, mtype)
    preds_dict, acc = eval_model(dataloaders=dataloaders,
                                 covid_model=covid_ft,
                                 nor_pneu_model=nor_pneu_ft,
                                 criterion=criterion,
                                 batch_size=batch_size,
                                 device=device,
                                 out_dir=out_dir,
                                 mtype=mtype)

--------------------------------------- 

Evaluation Result
----------
Val Acc: 0.9533

데이터셋A: 300
Covid Recall: 93.0
Covid Precision: 98.94
Covid ACC: 97.33000000000001
Covid F1 score: 95.88

데이터셋B: 300
Normal Recall: 97.0
Normal Precision: 91.51
Normal ACC: 96.0
Normal F1 score: 94.18

데이터셋C: 300
Pneumonia Recall: 96.0
Pneumonia Precision: 96.0
Pneumonia ACC: 97.33000000000001
Pneumonia F1 score: 96.0
Training complete in 0m 16s


## Verify prediction results

In [8]:
cases = [(i, j) for i in range(3) for j in range(3)]
preds_cols = ('covid', 'normal', 'pneumonia')
preds_stat_cols = ('covid-mean', 'covid-std', 'normal-mean', 'normal-std',
                   'pneumonia-mean', 'pneumonia-std', 'count')

preds_dfs = []
preds_stat_df = pd.DataFrame([], columns=preds_stat_cols)
for case in cases:
    preds_df = pd.DataFrame(preds_dict[case], columns=preds_cols)

    preds_stat = []
    for preds_col in preds_cols:
        preds_stat.extend((preds_df[preds_col].mean(), preds_df[preds_col].std()))
    preds_stat.append(preds_df[preds_col].shape[0])

    if np.nan not in preds_stat:
        preds_stat_df = preds_stat_df.append(pd.DataFrame([preds_stat], columns=preds_stat_cols, index=(str(case),)))
    
preds_stat_df

,covid-mean,covid-std,normal-mean,normal-std,pneumonia-mean,pneumonia-std,count
"(0, 0)",0.928496,0.149504,0.040154,0.136334,0.105670,0.178705,93
"(0, 2)",0.996965,NaN,0.235122,NaN,0.025455,NaN,1
"(1, 0)",0.556875,0.402212,0.973347,0.021636,0.012013,0.017553,6
"(1, 1)",0.019653,0.065932,0.953397,0.087992,0.034025,0.088233,97
"(1, 2)",0.000008,0.000010,0.922776,0.053992,0.014983,0.010975,3
"(2, 0)",0.077755,NaN,0.166851,NaN,0.199409,NaN,1
"(2, 1)",0.055649,0.079911,0.505253,0.239999,0.274283,0.232358,3
"(2, 2)",0.044035,0.143221,0.018581,0.087356,0.922603,0.173569,96
